# (Prototype) COVID Dashboard Map

This is a prototype COVID Dashboard for testing displaying COVID data on maps (initially testing using the folium library).

In [1]:
# This forces a reload of any external library file if it changes.  
# Useful when developing external libraries since otherwise Jupyter 
# will not re-import any library without restarting the python kernel.

%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
import branca
import folium
import time

# Import COVID IO routines from external python libraries
import COVIDlib.data_IO as COVID_IO
import COVIDlib.dashboard_IO as COVID_Dash

import ipyleaflet as lf
import json

## Define variables of interest below
data_dir = 'our_data/'    # Data directory for the COVID datafiles

In [3]:
# Load all the dataframes into memory
print("Loading and Preprocessing COVID Dataset ... ", end='')

start= time.perf_counter()
# Retrieve John Hopkins dataframes and add "rates" of deaths/infections
(JH_state_df, JH_cnty_df) = COVID_IO.PtoCDRDataFrames()
JH_state_df = COVID_Dash.cleanJHdata(JH_state_df)
JH_cnty_df = COVID_Dash.cleanJHdata(JH_cnty_df)

# Construct dictionary of FIPS values by placename
FIPSd = COVID_Dash.build_fipsdict(JH_cnty_df, JH_state_df)

# Retrieve Apple Mobility Dataframe
(aapl_cnty_df, aapl_state_df) = COVID_IO.PtoAAPLMobilityDataFrames()
COVID_Dash.cleanAAPLdata(aapl_cnty_df)
COVID_Dash.cleanAAPLdata(aapl_state_df)

# Retrieve Google Mobility Dataframe
(goog_cnty_df, goog_state_df) = COVID_IO.PtoGOOGMobilityDataFrames()

# Retrieve IMHE Dataframes
(summary_df, hospitalization_df) = COVID_IO.PtoIMHEDataFrames()

end= time.perf_counter()

print(f"Done ({end-start:0.2f} sec)")

Loading and Preprocessing COVID Dataset ... Done (1.36 sec)


In [4]:
## This initial test version is based on a prototype seen at https://python-graph-gallery.com/292-choropleth-map-with-folium/

# Location of JSON with the shapes of the US states (note in this JSON, the "id" field [used to match to data] consists of 2 letter postal codes [e.g. MN or ND])
state_geo = './folium_jsons/us-states.json'

# Location of JSON with the shapes of the US counties (only has four keys, none are ID, don't know if it is just stored in FIPS number order)
county_geo = './folium_jsons/us_counties_20m_topo.json'

# Grab only the state data and then redefine columns to have only last day's data
state_data = JH_state_df[JH_state_df['state'] != 'United States'].copy()

state_data['ConfirmedRate'] = np.array(state_data['ConfirmedRate'].to_list())[:,-1].tolist()
state_data['DeathsRate'] = np.array(state_data['DeathRate'].to_list())[:,-1].tolist()
state_data['dConfirmedRate'] = np.array(state_data['dConfirmedRate'].to_list())[:,-1].tolist()
state_data['dDeathsRate'] = np.array(state_data['dDeathsRate'].to_list())[:,-1].tolist()
state_data['d2ConfirmedRate'] = np.array(state_data['d2ConfirmedRate'].to_list())[:,-1].tolist()
state_data['d2DeathsRate'] = np.array(state_data['d2DeathsRate'].to_list())[:,-1].tolist()


In [5]:
county_data = JH_cnty_df.set_index('FIPS').copy()  # Index by FIPS in this copy of the county data

# only retain the most recent value in these columns
county_data['ConfirmedRate'] = np.array(county_data['ConfirmedRate'].to_list())[:,-1].tolist()
county_data['DeathsRate'] = np.array(county_data['DeathRate'].to_list())[:,-1].tolist()
county_data['dConfirmedRate'] = np.array(county_data['dConfirmedRate'].to_list())[:,-1].tolist()
county_data['dDeathsRate'] = np.array(county_data['dDeathsRate'].to_list())[:,-1].tolist()
county_data['d2ConfirmedRate'] = np.array(county_data['d2ConfirmedRate'].to_list())[:,-1].tolist()
county_data['d2DeathsRate'] = np.array(county_data['d2DeathsRate'].to_list())[:,-1].tolist()


## County-level maps possible

- I have not figured out how to plot/display the colormap
- I figured out how to **LOCK** down zooming and panning the maps, so this isn't interactive, see the example bere.
- There is supposed to be a way to use the county level JSON file in the choropleth command, couldn't figure it out.
- The IDs for the county level data are FIPS (allthough they may be forced to be 5 digits with padding with zeros, eg what we call 1001 they call 01001).

In [6]:
# Define a styling function
def style_function(feature):
    data = county_data_series.get(int(feature['id'][-5:]), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if data is None else colorscale(data)
    }

# load JSON data as strings
f = open(county_geo, "r")
json_data  = f.read()

# Pick the data series to plot
county_data_series = county_data['dConfirmedRate']

# Set ColorScale using a maximum value of 5 * the median.  It would be nice if we could have 
# colorbar to indicate the rate or a tooltip (hover over a county to get the name and infection rate)
colorscale = branca.colormap.linear.YlOrRd_09.scale(county_data_series.min(), county_data_series.max())

# Build a County Level Map of Infection rates by styling each county by its infection level
ZoomLevel = 5
infection_countymap = folium.Map(
    location=[45, -104],
    tiles='cartodbpositron',
    zoom_start=ZoomLevel, min_zoom=ZoomLevel, max_zoom=ZoomLevel, zoom_control=False, no_touch=True,  # Locks the Zoom level, but allows panning
    dragging=False, doubleClickZoom=False,  # Locks Dragging
    control_scale=True,
    legend_name='Total New Infections in last 24 hours (per 100,000 people)'
)

# Maps out json geodata file using style_function (which here looks up county_data_series) to set the value
folium.TopoJson(
    json.loads(json_data),
    'objects.us_counties_20m',
    style_function=style_function
).add_to(infection_countymap )


infection_countymap 

##

In [7]:
##
## Try calling choropleth command uing same topojson object as above

# Set ColorScale using a maximum value of 5 * the median.  It would be nice if we could have 
# colorbar to indicate the rate or a tooltip (hover over a county to get the name and infection rate)
colorscale = branca.colormap.linear.YlOrRd_09.scale(county_data_series.min(), county_data_series.max())

# Initialize the map
infection_countymap2 = folium.Map(location=[45, -104], zoom_start=4)

# Maps out json geodata file using style_function (which here looks up county_data_series) to set the value
topojson_obj = folium.TopoJson(
    json.loads(json_data),
    'objects.us_counties_20m',
    style_function=style_function
)

# Add the color for the chloropleth:
totinfections_map.choropleth(
 geo_data=county_geo,
 name='Total Infection Rate',
 data=county_data,
 columns=['postal', 'ConfirmedRate'],
 key_on='feature.id',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 topojson =topojson_obj,
 legend_name='New Infections in last 24 hours (per 100,000 people)'
)
folium.LayerControl().add_to(infection_countymap2)

# Display the map
infection_countymap2 

NameError: name 'totinfections_map' is not defined

## Total COVID Infection Rate Map

In [8]:
# Initialize the map
totinfections_map = folium.Map(location=[45, -104], zoom_start=4)

# Add the color for the chloropleth:
totinfections_map.choropleth(
 geo_data=state_geo,
 name='Total Infection Rate',
 data=state_data,
 columns=['postal', 'ConfirmedRate'],
 key_on='feature.id',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Total Infections (per 100,000 people)'
)
folium.LayerControl().add_to(totinfections_map)

# Display the map
totinfections_map

C:\Users\ljheb\anaconda3\lib\site-packages\folium\folium.py:415: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


## Total COVID Death Rate Map

In [9]:
# Initialize the map
totdeaths_map = folium.Map(location=[45, -104], zoom_start=4)

# Add the color for the chloropleth:
totdeaths_map.choropleth(
 geo_data=state_geo,
 name='Total Death Rate',
 data=state_data,
 columns=['postal', 'DeathsRate'],
 key_on='feature.id',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Total Deaths (per 100,000 people)'
)
folium.LayerControl().add_to(totdeaths_map)

# Display the map
totdeaths_map

## New COVID Infections Rate Map

In [10]:
# Initialize the map
infections_map = folium.Map(location=[45, -104], zoom_start=4)

# Add the color for the chloropleth:
infections_map.choropleth(
 geo_data=state_geo,
 name='New Infection Rate',
 data=state_data,
 columns=['postal', 'dConfirmedRate'],
 key_on='feature.id',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='New Infections in Last 24 Hours (per 100,000 people)'
)
folium.LayerControl().add_to(infections_map)

# Display the map
infections_map

## Total COVID Infection Rate Change Map

In [11]:
# Initialize the map
delta_infections_map = folium.Map(location=[45, -104], zoom_start=4
                                 )

# Add the color for the chloropleth:
delta_infections_map.choropleth(
 geo_data=state_geo,
 name='Change Rate of New Infections',
 data=state_data,
 columns=['postal', 'd2ConfirmedRate'],
 key_on='feature.id',
 fill_color='YlOrRd',
 fill_opacity=0.7,
 line_opacity=0.2,
 legend_name='Change Rate of New Infections (per 100,000 people)'
)
folium.LayerControl().add_to(delta_infections_map)

# Display the map
delta_infections_map

In [53]:
# ipyleaflet map testing below
# Luke's to do: get JSON data to work, add choropleth

In [54]:
center = [38.0, -93.0]
zoom = 3.9

# Creating the map
ipyleaflet_map = lf.Map(basemap=lf.basemaps.OpenStreetMap.BlackAndWhite, center = center, zoom = zoom)

In [55]:
# Reading the JSON file
with open(county_geo) as file:
    county_json = json.load(file)
json_data = lf.GeoJSON(data=county_json)

# Adding the JSON data to the map
ipyleaflet_map.add_layer(json_data)

In [56]:
ipyleaflet_map

Map(center=[38.0, -93.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_ou…